In [3]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
#import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union
from pydantic.config import ConfigDict

from src.models import *

from src.utilities import *
from src.bubble import *

import openai
import instructor

#openai.api_key = os.environ["OPENAI_API_KEY"]


In [4]:
prompts_path = "Prompts/fr/"

In [6]:
with open(prompts_path+'prompt_company_informations.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_company_informations = template.invoke(context)

class CompanyContext(BaseModel):
    context: str = Field(description="Contexte de l'entreprise.")

    def __str__(self):
        return self.context

res = get_analysis(prompt_company_informations, CompanyContext)
context['context'] = res.context
print(res)

Darty est une entreprise française spécialisée dans la vente d'appareils électroménagers, d'équipements audiovisuels et de produits informatiques. Fondée en 1957, la société a connu une croissance constante et s'est imposée comme un acteur majeur du secteur en France. Récemment, Darty a été racheté par le groupe Fnac Darty en 2017, ce qui a permis à l'enseigne de renforcer sa position sur le marché national et de s'ouvrir à de nouvelles opportunités de développement. Les concurrents directs de Darty sont Boulanger, But, et Conforama.


In [8]:
with open(prompts_path+'prompt_categories_creation.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_categories_creation = template.invoke(context)

class InsightSubCategory(BaseModel):
    name: str = Field(description="Nom de la sous-catégorie.")
    description: str = Field(description="Phrase de description de la sous-catégorie.")
    id: str
    def __str__(self):
        return '\n- ' + self.name + ": " + self.description

class InsightCategory(BaseModel):
    name: str = Field(description="Nom de la catégorie.")
    #description: str = Field(description="Phrase de description de la catégorie.")
    sub_categories: List[InsightSubCategory] = Field(description="Sous-catégories.")
    id: str

    def __str__(self):
        return self.name + " : " + ''.join([str(x) for x in self.sub_categories])

class InsightCategories(BaseModel):
    categories: List[InsightCategory] = Field(description="Catégories parmi lesquels les insights pourront être classifiées.")

    def __str__(self):
        return '\n\n'.join([str(x) for x in self.categories])

res = get_analysis(prompt_categories_creation, InsightCategories)
print(res)

Satisfaction globale : 
- Note globale: Indique le niveau de satisfaction globale du client vis-à-vis de Darty.
- Commentaire général: Permet au client de s'exprimer librement sur son expérience chez Darty.

Expérience en magasin : 
- Propreté du magasin: Évalue la propreté du magasin et des rayons.
- Achalandage: Indique si le magasin était trop achalandé ou si les files d'attente étaient trop longues.
- Amabilité du personnel: Évalue la gentillesse et la serviabilité du personnel.
- Disponibilité des produits: Indique si les produits recherchés par le client étaient disponibles en magasin.
- Délais d'attente en caisse: Évalue le temps d'attente en caisse.

Qualité des produits : 
- Fraîcheur des produits: Évalue la fraîcheur des produits frais et périssables.
- Variété des produits: Indique si le choix de produits est suffisant et répond aux attentes du client.
- Qualité des produits MDD: Évalue la qualité des produits de la marque Darty.

Prix : 
- Niveau des prix: Indique si le cli

In [9]:
for categorie in tqdm(res.categories):
    categorie.id = bubble_client.create(
        "Category",
        {
            "Company": COMPANY_ID,
            "Name": categorie.name,
            "Project": PROJECT_ID,
        })
    
    for sub_categorie in categorie.sub_categories:
        sub_categorie.id = bubble_client.create(
            "SubCategory",
            {
                "Company": COMPANY_ID,
                "Parent_category": categorie.id,
                "Name": sub_categorie.name,
                "Project": PROJECT_ID,
                "Description": sub_categorie.description,
            })

  0%|          | 0/10 [00:00<?, ?it/s]